In [12]:
import requests
from bs4 import BeautifulSoup
import re
import math
import requests
import pandas as pd 

In [13]:

#_____________________________________________________________________________________________
# extracting links

# generate the group links (for 333 pages or less)
def generate_page_links(base_link, total_pages):
    page_links = [base_link]
    for p in range(2, total_pages + 1):
        page_links.append(base_link + "&page=" + str(p))
    return page_links


# extract all the linkes items for sale 
def extract_links_from_page(session, page_link):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = session.get(page_link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    links = []
    for tag_a in soup.find_all('a', class_="card__title-link", href=True):
        links.append(tag_a['href'])
    return links

# filter "new real estate project" links
def filter_links(links):
    return [link for link in links if "/new-real-estate-project-" not in link]

# save the links in a csv file
def save_links_to_csv(links, file_path):    # save the files in only 1 file
    df = pd.DataFrame(links, columns=['URL'])
    file_name = f'{file_path}/all_links.csv'  
    df.to_csv(file_name, index=False, encoding='utf-8')
    print(f"All links saved in {file_name}")

# def save_links_to_csv(links, file_path, num_files):   # save the files in 10 file
#     links_per_file = math.ceil(len(links) / num_files)
    
#     for i in range(num_files):
#         start_index = i * links_per_file
#         end_index = min((i + 1) * links_per_file, len(links))
#         links_chunk = links[start_index:end_index]

#         df = pd.DataFrame(links_chunk, columns=['URL'])
#         file_name = f'{file_path}/links_{i+1}.csv'
#         df.to_csv(file_name, index=False, encoding='utf-8')



# calling functions to extract links

base_link = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2'
total_pages = 333

# calling to generate 333 grope page links
page_links = generate_page_links(base_link, total_pages)

# extract the links of sale items from 333 group pages (extracting +10000 links)
session = requests.Session()
all_links = []
for page_link in page_links[:3]:  # we can choose wor on 333 pages or less  ///////////////////////////////////
    all_links.extend(extract_links_from_page(session, page_link))

# calling to filter the "new real estate project" links
filtered_links = filter_links(all_links)

# calling to save the extracted links to a csv file
all_saved_linked = save_links_to_csv(filtered_links, 'C:\\Users\\becod\\AI\\my-projects\\Majid_immoeliza_cleanup\\Links')

print("the links saved in csv file")



All links saved in C:\Users\becod\AI\my-projects\Majid_immoeliza_cleanup\Links/all_links.csv
the links saved in csv file


In [14]:
#_____________________________________________________________________________________________
#  Extracting detailes

# request to URL to extract tht HTML content
def get_soup(url):
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"}
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")
    return soup

# extracting Property ID
def get_property_id(soup):
    try:
        html = soup.find("meta", {'property': "og:url"}).get('content')
        html_list = html.split("/")
        print("code is running")
        return html_list[-1]
    except Exception as e:
        #print(f"Error in Property_ID: {e}")
        return None
    
# extracting Postal code
def get_postal_code(html_list):
    try:
        return html_list[-2]
    except Exception as e:
        #print(f"Error in postal_code : {e}")
        return None

# extracting Locality (city)
def get_locality(html_list):
    try:
        return html_list[-3]
    except Exception as e:
        #print(f"Error in locality : {e}")
        return None

# extracting price as string
def get_price(soup):
    try:
        home_meta_info = soup.find_all("div", {'class': 'grid__item desktop--9'})
        price = home_meta_info[0].find("p", {'class': 'classified__price'}).find_all('span', {'class':'sr-only'})[0].text.strip()
        price = re.sub(r'[^\d.,\-+]', '', price)
        return str(price)    # convert to str
    except Exception as e:
        #print(f"Error in price : {e}")
        return None

# Type of property
def get_property_type(html_list):
    try:
        return html_list[-5]
    except Exception as e:
        #print(f"Error in Type_of_property: {e}")
        return None

# extracting number of bedrooms as string
def get_bedrooms(soup):
    try:
        home_prop_info = soup.find_all("div", {'class': 'text-block__body'})[0].find_all("div", {'class': 'overview__column'})
        bed_rooms =home_prop_info[0].find_all("div", {'class': 'overview__item'})[0].find_all('span', {'class':'overview__text'})[0].text.strip()
        bed_rooms = re.search(r'\d+', bed_rooms)
        return str(bed_rooms.group()) if bed_rooms else None    # convert to str
    except Exception as e:
        #print(f"Error in bed_rooms : {e}")
        return None

# extracting Living area size  as string
def get_living_area(soup):
    try:
        home_prop_info = soup.find_all("div", {'class': 'text-block__body'})[0].find_all("div", {'class': 'overview__column'})
        space = home_prop_info[1].find_all("div", {'class': 'overview__item'})[0].find_all('span', {'class':'overview__text'})[0].text.strip()
        space = re.findall(r'\d+', space)[0]
        return str(space)  # convert to str
    except Exception as e:
        #print(f"Error in space: {e}")
        return None

# extracting kitchen type
def get_kitchen(soup):
    try:
        kitchen_keywords = ('Kitchen type', 'Type of kitchen')
        kitchen_th = soup.find('th', string=lambda x: x and x.strip() in kitchen_keywords)
        if kitchen_th:
            kitchen = kitchen_th.find_next_sibling('td').contents[0].strip()
            return 1 if kitchen in ('Installed','Installed', 'Hyper equipped', 'USA  Hyper equipped','Semi equipped','USA hyper equipped') else 0
        return 0
    except Exception as e:
        #print(f"Error in kitchen: {e}")
        return None

# extracting state of building
def get_building_condition(soup):
    try:
        building_condition_header = soup.find('th', string=lambda x: x and x.strip() == 'Building condition').find_parent('tr')
        return building_condition_header.find('td', class_='classified-table__data').contents[0].strip()
    except Exception as e:
        #print(f"Error in building_condition: {e}")
        return None

# extracting number of frontages (facades) as string
def get_number_of_facades(soup):
    try:
        facade_keywords = re.compile(r'Number of (frontages|facades)', re.IGNORECASE)
        facades_th = soup.find('th', string=facade_keywords)
        return str(facades_th.find_next_sibling('td').contents[0].strip()) if facades_th else None  # convert to str
    except Exception as e:
        #print(f"Error in Number_of_facades : {e}")
        return None

# extracting furnished or not
def get_furnished(soup):
    try:
        Furnished = soup.find('th', string=lambda x: x and x.strip() == 'Furnished').find_next_sibling('td').contents[0].strip()
        return 1 if Furnished == 'Yes' else 0
    except Exception as e:
        #print(f"Error in Furnished: {e}")
        return 0

# extracting fire place state
def get_open_fire(soup):
    try:
        Open_fire = soup.find('th', string=lambda x: x and x.strip() == 'How many fireplaces?').find_next_sibling('td').contents[0].strip()
        return 1 if Open_fire else 0
    except Exception as e:
        #print(f"Error in Open_fire: {e}") 
        return 0

# extracting swimming pool state
def get_swimming_pool(soup):
    try:
        Swimming_pool = soup.find('th', string=lambda text: text and 'Swimming pool' in text.strip()).find_next_sibling('td').contents[0].strip()
        return 1 if Swimming_pool == 'Yes' else 0
    except Exception as e:
        #print(f"Error in Swimming_pool: {e}")
        return 0

# extracting garden state and garden size as string
def get_garden(soup):
    try:
        garden = soup.find('th', string=re.compile(r'^Garden.*')).find_next_sibling('td').contents[0].strip()
        return str(garden) if garden else None  # convert to str
    except Exception as e:
        #print(f"Error in garden: {e}")
        return None

# extracting trass status and trass size as string
def get_terrace(soup):
    try:
        Terrace = soup.find('th', string=re.compile(r'^Terrace.*')).find_next_sibling('td').contents[0].strip()
        return str(Terrace) if Terrace else None  # convert to str
    except Exception as e:
        #print(f"Error in Terrace: {e}")
        return None
    
# extracting public or online sale
def get_sale_type(soup):
    try:
        sale_keywords = ['public sale', 'online sale']       # looking for "public sale" "online sale" 
        sale_text = soup.find('body').text.lower()
        for keyword in sale_keywords:
            if keyword in sale_text:
                return keyword           # return it if its for onlune sale
        return None                       # if note, none
    except Exception as e:
        #print(f"Error in sale_type: {e}")
        return None


def get_sale_type(soup):
    try:
        sale_section = soup.find('span', class_='text-block__subtitle')      
        if sale_section:
            sale_text = sale_section.text.strip()  # remove spaces 
            if sale_text in ['Public sale', 'Online sale']:  # compare with text
                return sale_text       
        return None
    except Exception as e:
        return None

# calling all functions
def request_url(url):
    soup = get_soup(url)
    html_list = soup.find("meta", {'property': "og:url"}).get('content').split("/")
    
    property_details = {
        'Property ID': get_property_id(soup),
        'Postal code': get_postal_code(html_list),
        'Locality name': get_locality(html_list),
        'Price': get_price(soup),
        'Type of property': get_property_type(html_list),
        'Number of rooms': get_bedrooms(soup),
        'Living area': get_living_area(soup),
        'Equipped kitchen': get_kitchen(soup),
        'State of building': get_building_condition(soup),
        'Number of facades': get_number_of_facades(soup),
        'Furnished': get_furnished(soup),
        'Open fire': get_open_fire(soup),
        'Swimming pool': get_swimming_pool(soup),
        'Garden (m²)': get_garden(soup),
        'Terrace (m²)': get_terrace(soup),
        'Sale type': get_sale_type(soup)  # sale type
    }

    return property_details

#_____________________________________________________________________________________________
#  calling and saving


# reading input file (linkes)
def read_input_file(file_path):
    try:
        df = pd.read_csv(file_path)
        return df['URL'].tolist()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

# process the links and extract the details of sale items
def process_urls(url_list):
    property_details = []
    for url in url_list:
        property_details.append(request_url(url))
    return property_details

# saving the details of sale items
# def write_output_file(file_path, property_details):
#     try:
#         df_properties = pd.DataFrame(property_details)
#         df_properties.to_csv(file_path, index=False, encoding='utf-8')
#         print(f"CSV file created: {file_path}")
#     except Exception as e:
#         print(f"Error writing to file {file_path}: {e}")

# saving the details of sale items
def write_output_file(file_path, property_details):
    try:
        df_properties = pd.DataFrame(property_details)
        
        print(df_properties.head())
        
        df_properties.to_csv(file_path, index=False, encoding='utf-8')
        print(f"CSV file created: {file_path}")
    except Exception as e:
        print(f"Error writing to file {file_path}: {e}")




In [ ]:
# Main function to process all files
def process_files(input_file_template, output_file_path):   # calling to save all details in only 1 file
    url_list = read_input_file(input_file_template)  # read the files from linkes 
    property_details = process_urls(url_list)   # take the links and extract the details  
    write_output_file(output_file_path, property_details)   # save all details in a csv file


# def process_files(num_files, input_file_template, output_file_template):
#     for i in range(1, num_files + 1):  # Loop from 1 to num_files
#         input_file_path = input_file_template.format(i)
#         output_file_path = output_file_template.format(i)      
#         # reading the input file and extract the links
#         url_list = read_input_file(input_file_path)       
#         # process the links and extract the detail of items
#         property_details = process_urls(url_list)       
#         # save the extracted detail in output file
#         write_output_file(output_file_path, property_details)

# setting the paths and number of files
input_file_path = "C:\\Users\\becod\\AI\\my-projects\\Majid_immoeliza_cleanup\\Links\\all_links.csv"
output_file_path_template = "C:\\Users\\becod\\AI\\my-projects\\Majid_immoeliza_cleanup\\Details\\property_details.csv"

# calling the main function 
process_files(input_file_path, output_file_path_template)


In [16]:
# cleaning
# __________________________________________________________________________

# read CSV file
def read_csv(file_path):
    return pd.read_csv(file_path)

# __________________________________________________________________________

# 1. remove duplicate rows
def remove_duplicates(data):
    return data.drop_duplicates()

# 2. remove properties with duplicate 'Property ID' but different values
def remove_duplicate_properties(data):
    duplicate_properties = data.groupby('Property ID').filter(lambda x: len(x) > 1)
    return data[~data['Property ID'].isin(duplicate_properties['Property ID'])]

# 3.1. clean 'Locality name' column by removing extra characters
def clean_locality_name(name):
    name = name.replace('%C', 'e')               # replace  %C with  e (it was originally è)
    name = name.replace('%27', "'")              # replace  %27 with  '
    name = name.replace('%20', ' ')              # replace  %20 with  space
    name = re.sub(r'%20%28.*?%29', '', name)     # remove characters between %20%28 and %29 
    name = re.sub(r'[0-9\(\)%]', '', name)       # remove any numbers and special characters (like parenthes and %)
    name = re.sub(r'-\d+$', '', name)            # remove numbers and - if a city name ended with (-number)
    name = name.lower().title()                  # convert to lowercase and capitalize the first letter of each word # for some cities with several word name 
    return name.strip()                          # remove any extra spaces

# 3.2. Apply the cleaning function to the 'Locality name' column in the dataset
def clean_locality_names_column(df):
    df['Locality name'] = df['Locality name'].apply(clean_locality_name)
    return df

# 4.remove rows if 'Sale type' value is 'online sale' or 'public sale'
def remove_sale_type(data):
    return data[~data['Sale type'].isin(['online sale', 'public sale'])]


# __________________________________________________________________________

# save the cleaned DataFrame to a CSV file
def save_to_csv(data, output_path):
    data.to_csv(output_path, index=False)
    print(f"Data saved to {output_path}")

# __________________________________________________________________________

# main function to orchestrate the cleaning process
def main():
    file_path = 'C:\\Users\\becod\\AI\\my-projects\\Majid_immoeliza_cleanup\\Details\\property_details.csv'
  
    output_file_path = 'C:\\Users\\becod\\AI\\my-projects\\Majid_immoeliza_cleanup\\Cleaned\\all_results_cleanedup.csv'
    
    # Read the CSV file
    data = read_csv(file_path)
    cd
    data_final = remove_sale_type(data_final)
    
    # Save the cleaned data to a new CSV
    save_to_csv(data_final, output_file_path)

# Call the main function
main()


Data saved to C:\Users\becod\AI\my-projects\Majid_immoeliza_cleanup\Cleaned\all_results_cleanedup.csv


In [ ]:
# تمیزکاری داده‌ها
# __________________________________________________________________________

# خواندن فایل CSV
def read_csv(file_path):
    # اینجا از کتابخانه pandas برای خواندن فایل CSV و بارگذاری آن در یک DataFrame استفاده شده است
    return pd.read_csv(file_path)

# __________________________________________________________________________

# 1. حذف ردیف‌های تکراری
def remove_duplicates(data):
    # اینجا از متد drop_duplicates در pandas برای حذف ردیف‌های تکراری از DataFrame استفاده شده است
    return data.drop_duplicates()

# 2. حذف املاک با 'Property ID' تکراری اما مقادیر متفاوت
def remove_duplicate_properties(data):
    # اینجا از متد groupby و filter در pandas برای پیدا کردن 'Property ID' های تکراری استفاده شده است
    # همچنین از isin و ایندکس‌گذاری pandas برای فیلتر کردن DataFrame استفاده شده است
    duplicate_properties = data.groupby('Property ID').filter(lambda x: len(x) > 1)
    return data[~data['Property ID'].isin(duplicate_properties['Property ID'])]

# 3.1. پاکسازی ستون 'Locality name' با حذف کاراکترهای اضافی
def clean_locality_name(name):
    # اینجا از pandas به طور مستقیم استفاده نشده است، اما این تابع بعداً بر روی یک ستون pandas اعمال می‌شود
    name = name.replace('%C', 'e')               # جایگزینی %C با e (اصالتاً è بود)
    name = name.replace('%27', "'")              # جایگزینی %27 با '
    name = name.replace('%20', ' ')              # جایگزینی %20 با فاصله
    name = re.sub(r'%20%28.*?%29', '', name)     # حذف کاراکترها بین %20%28 و %29
    name = re.sub(r'[0-9\(\)%]', '', name)       # حذف هر عدد و کاراکترهای خاص مانند پرانتز و %
    name = re.sub(r'-\d+$', '', name)            # حذف اعداد و - اگر نام شهر با (-عدد) تمام شود
    name = name.lower().title()                  # تبدیل به حروف کوچک و سپس بزرگ کردن اولین حرف هر کلمه
    return name.strip()                          # حذف فاصله‌های اضافی

# 3.2. اعمال تابع پاکسازی بر روی ستون 'Locality name' در دیتاست
def clean_locality_names_column(df):
    # اینجا از متد apply در pandas برای اعمال تابع پاکسازی بر روی ستون 'Locality name' استفاده شده است
    df['Locality name'] = df['Locality name'].apply(clean_locality_name)
    return df

# 4. حذف ردیف‌ها اگر مقدار 'Sale type' برابر با 'online sale' یا 'public sale' باشد
def remove_sale_type(data):
    # اینجا از متد isin در pandas برای فیلتر کردن ردیف‌هایی که 'Sale type' برابر با 'online sale' یا 'public sale' است استفاده شده است
    return data[~data['Sale type'].isin(['online sale', 'public sale'])]

# __________________________________________________________________________

# ذخیره کردن DataFrame تمیز شده در یک فایل CSV
def save_to_csv(data, output_path):
    # اینجا از متد to_csv در pandas برای ذخیره DataFrame به یک فایل CSV استفاده شده است
    data.to_csv(output_path, index=False)
    print(f"Data saved to {output_path}")

# __________________________________________________________________________

# تابع اصلی برای مدیریت فرآیند تمیزکاری
def main():
    file_path = 'C:\\Users\\becod\\AI\\my-projects\\Majid_immoeliza_cleanup\\Details\\property_details.csv'
    output_file_path = 'C:\\Users\\becod\\AI\\my-projects\\Majid_immoeliza_cleanup\\Cleaned\\all_results_cleanedup.csv'
    
    # 1. خواندن فایل CSV به یک DataFrame
    # از متد read_csv pandas استفاده شده است
    data = read_csv(file_path)
    
    # 2. حذف ردیف‌های تکراری با استفاده از متد drop_duplicates pandas
    data_cleaned = remove_duplicates(data)
    
    # 3. حذف املاک با 'Property ID' تکراری اما مقادیر متفاوت با استفاده از متد groupby، filter، isin و فیلتر کردن در pandas
    data_final = remove_duplicate_properties(data_cleaned)
    
    # 4. پاکسازی ستون 'Locality name' با اعمال تابع تمیزسازی با استفاده از متد apply pandas
    data_final = clean_locality_names_column(data_final)

    # 5. حذف ردیف‌ها با 'online sale' یا 'public sale' در 'Sale type' با استفاده از متد isin pandas
    data_final = remove_sale_type(data_final)
    
    # 6. ذخیره DataFrame تمیز شده به یک فایل CSV جدید با استفاده از متد to_csv pandas
    save_to_csv(data_final, output_file_path)

# فراخوانی تابع اصلی
main()
